<a href="https://colab.research.google.com/github/sravan1320/RAG/blob/main/LLAMA2_RAG_ChromaDB_Chat_With_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# !pip install transformers langchain torch constants chromadb accelerate sentence-transformers pypdf pdfminer.six

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) N
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
from chromadb.config import Settings
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory='db',
        anonymized_telemetry=False
)

In [18]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os


persist_directory = "db"

for root, dirs, files in os.walk("docs"):
    for file in files:
        if file.endswith(".pdf"):
            print(file)
            loader = PyPDFLoader(os.path.join(root, file))
documents = loader.load()
print("splitting into chunks")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
#create embeddings here
print("Loading LlamaCppEmbeddings model")
# checkpoint = "MBZUAI/LaMini-T5-738M"
# print(f"Checkpoint path: {checkpoint}")  # Add this line for debugging
# embeddings = LlamaCppEmbeddings(model_name=checkpoint)
# specify embedding model (using huggingface sentence transformer)
checkpoint = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=checkpoint, model_kwargs=model_kwargs)

#create vector store here
print(f"Creating embeddings. May take some minutes...")
db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory) #, client_settings=CHROMA_SETTINGS)
db.persist()
db=None

print(f"Ingestion complete! You can now run privateGPT.py to query your documents")

2018-Annual-Report.pdf
splitting into chunks
Loading LlamaCppEmbeddings model
Creating embeddings. May take some minutes...
Ingestion complete! You can now run privateGPT.py to query your documents


##Chroma Load from Disk Example

In [20]:
# load from disk
query = "what is quaterly results (unaudited)?"
db3 = Chroma(persist_directory="db", embedding_function=embeddings)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Note 11—QUAR TERL YRESUL TS(UNAUDITED)
Thefollowing tables contain selected unaudited statement ofoperations information foreach quarter of2017 and2018.
Thefollowing information reflects allnormal recurring adjustments necessary forafairpresentation oftheinformation forthe
periods presented. Theoperating results foranyquarter arenotnecessarily indicative ofresults foranyfuture period. Our


In [3]:
import os
import base64
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import textwrap
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

device = torch.device('cuda')

checkpoint = "MBZUAI/LaMini-T5-738M"
print(f"Checkpoint path: {checkpoint}")  # Add this line for debugging
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map=device,
    torch_dtype=torch.float32
)

persist_directory = "db"

def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p= 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

def qa_llm():
    llm = llm_pipeline()
    embedding_checkpoint = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {"device": "cuda"}
    embeddings = HuggingFaceEmbeddings(model_name=embedding_checkpoint, model_kwargs=model_kwargs)
    # embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma(persist_directory="db", embedding_function = embeddings) #, client_settings=CHROMA_SETTINGS)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        return_source_documents=True
    )
    return qa

def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    return answer

Checkpoint path: MBZUAI/LaMini-T5-738M


In [4]:
query = "what is quaterly results (unaudited)?"
process_answer(query)

'The provided text does not provide information about what quartary results are.'

In [8]:
qa = qa_llm()
generated_text = qa(query)
print(generated_text)

{'query': 'what is quaterly results (unaudited)?', 'result': 'The provided context does not give a clear answer to what quaterly results (unaudited) are.', 'source_documents': [Document(page_content='Note 11—QUAR TERL YRESUL TS(UNAUDITED)\nThefollowing tables contain selected unaudited statement ofoperations information foreach quarter of2017 and2018.\nThefollowing information reflects allnormal recurring adjustments necessary forafairpresentation oftheinformation forthe\nperiods presented. Theoperating results foranyquarter arenotnecessarily indicative ofresults foranyfuture period. Our', metadata={'page': 76, 'source': 'docs/2018-Annual-Report.pdf'}), Document(page_content='reliability offinancial reporting andthepreparation offinancial statements forexternal purposes inaccordance with generally\naccepted accounting principles. Acompany’ sinternal control over financial reporting includes those policies andprocedures\nthat(1)pertain tothemaintenance ofrecords that,inreasonable detail